### Create the staging layer of the air travel lakehouse

#### Declare variables used throughout

In [1]:
project_id = "cs378-fa2025"
dataset = "air_travel_stg"
region = "us-central1"

#### Create staging dataset

In [ ]:
from google.cloud import bigquery

bq_client = bigquery.Client()

dataset_id = bigquery.Dataset(f"{project_id}.{dataset}")
dataset_id.location = region
resp = bq_client.create_dataset(dataset_id, exists_ok=True)
print("Created dataset {}.{}".format(bq_client.project, resp.dataset_id))

Created dataset cs378-fa2025.air_travel_stg


### `aircrafts` table

In [2]:
%%bigquery
select * from air_travel_raw.aircrafts
where iata_code like '%\\N'
or icao_code like '%\\N'
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,aircraft_name,iata_code,icao_code,_data_source,_load_time
0,Airbus A330,330,\N,openflights,2025-09-06 02:42:39.189193+00:00
1,Boeing 757,757,\N,openflights,2025-09-06 02:42:39.189193+00:00
2,Boeing 747,747,\N,openflights,2025-09-06 02:42:39.189193+00:00
3,Beechcraft Baron,\N,BE58,openflights,2025-09-06 02:42:39.189193+00:00
4,Tupolev Tu-144,\N,T144,openflights,2025-09-06 02:42:39.189193+00:00


##### Replace `'\\N'` with null

In [ ]:
%%bigquery
select aircraft_name, iata_code,
case icao_code when '\\N' then null else icao_code end as icao_code,
  _data_source, _load_time
from air_travel_raw.aircrafts

Query is running:   0%|          |

Downloading:   0%|          |

,aircraft_name,iata_code,icao_code,_data_source,_load_time
0,Antonov AN-28,A28,AN28,openflights,2025-09-06 02:42:39.189193+00:00
1,Boeing 747-100,741,B741,openflights,2025-09-06 02:42:39.189193+00:00
2,Boeing 747-8,748,B748,openflights,2025-09-06 02:42:39.189193+00:00
3,Cessna Citation Excel,CNJ,C56X,openflights,2025-09-06 02:42:39.189193+00:00
4,Boeing 737 MAX 7,7M7,B37M,openflights,2025-09-06 02:42:39.189193+00:00
...,...,...,...,...,...
241,Airbus A330-300,333,A333,openflights,2025-09-06 02:42:39.189193+00:00
242,Boeing 737,737,None,openflights,2025-09-06 02:42:39.189193+00:00
243,Boeing 757-300,753,B753,openflights,2025-09-06 02:42:39.189193+00:00
244,Bombardier CS100,CS1,BCS1,openflights,2025-09-06 02:42:39.189193+00:00


##### Create staging table

In [ ]:
%%bigquery
create or replace table air_travel_stg.aircrafts as
  select aircraft_name as name,
  case iata_code when '\\N' then null else iata_code end as iata,
  case icao_code when '\\N' then null else icao_code end as icao,
  _data_source,
  _load_time
  from air_travel_raw.aircrafts;

Query is running:   0%|          |

""


##### Check the output

In [ ]:
%%bigquery
select * from air_travel_stg.aircrafts
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,name,iata,icao,_data_source,_load_time
0,Boeing 757,757,None,openflights,2025-09-06 02:42:39.189193+00:00
1,Boeing 747,747,None,openflights,2025-09-06 02:42:39.189193+00:00
2,BAe 146,146,None,openflights,2025-09-06 02:42:39.189193+00:00
3,Boeing 767,767,None,openflights,2025-09-06 02:42:39.189193+00:00
4,Embraer 175,E75,None,openflights,2025-09-06 02:42:39.189193+00:00


### `airlines` table

In [ ]:
%%bigquery
select * from air_travel_raw.airlines
where alias like '%\\N'
or country like '%\\N'
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,airline_id,name,alias,iata,icao,callsign,country,active,_data_source,_load_time
0,102,Aeroenlaces Nacionales,\N,None,AEN,None,Mexico,N,openflights,2025-09-05 17:02:52.385895+00:00
1,128,Alliance,\N,None,AFJ,JAMBO,Uganda,N,openflights,2025-09-05 17:02:52.385895+00:00
2,147,Air-Angol,\N,None,NGO,AIR ANGOL,Angola,N,openflights,2025-09-05 17:02:52.385895+00:00
3,257,Aktjubavia,\N,None,AKB,KARAB,Kazakhstan,N,openflights,2025-09-05 17:02:52.385895+00:00
4,270,Aeroalas Colombia,\N,None,ALE,AEROALAS,Colombia,N,openflights,2025-09-05 17:02:52.385895+00:00


##### Replace `'\\N'` with null

In [ ]:
%%bigquery
select airline_id as id,
name,
case alias when '\\N' then null else alias end as alias,
iata, icao, callsign,
case country when '\\N' then null else country end as country,
active, _data_source, _load_time
from air_travel_raw.airlines;

Query is running:   0%|          |

Downloading:   0%|          |

,id,name,alias,iata,icao,callsign,country,active,_data_source,_load_time
0,128,Alliance,None,None,AFJ,JAMBO,Uganda,N,openflights,2025-09-05 17:02:52.385895+00:00
1,465,Air Tanzania,None,TC,ATC,TANZANIA,Tanzania,Y,openflights,2025-09-05 17:02:52.385895+00:00
2,577,Alexandair,None,None,AXN,ALEXANDROS,Greece,N,openflights,2025-09-05 17:02:52.385895+00:00
3,647,Air Scorpio,None,None,SCU,SCORPIO UNIVERS,Bulgaria,N,openflights,2025-09-05 17:02:52.385895+00:00
4,1138,Air Parabet,None,None,PBT,PARABET,Bangladesh,N,openflights,2025-09-05 17:02:52.385895+00:00
...,...,...,...,...,...,...,...,...,...,...
6157,5126,Trans America,None,None,CLR,CLINTON AIRWAYS,United States,N,openflights,2025-09-05 17:02:52.385895+00:00
6158,5166,Trans Air,None,None,MUI,MAUI,United States,N,openflights,2025-09-05 17:02:52.385895+00:00
6159,5389,WSI Corporation,None,None,XWS,None,United States,N,openflights,2025-09-05 17:02:52.385895+00:00
6160,5434,Westward Airways,None,CN,WWD,WESTWARD,United States,N,openflights,2025-09-05 17:02:52.385895+00:00


##### Replace `\\N` and `''` with null

In [ ]:
%%bigquery
select airline_id as id,
name,
case alias
  when '\\N' then null
  else alias
  end as alias,
case iata
  when '' then null
  else iata
  end as iata,
case icao
  when '' then null
  else icao
  end as icao,
case callsign
  when '\\N' then null
  else callsign
  end as callsign,
case country
  when '\\N' then null
  else country end as country,
active, _data_source, _load_time
from air_travel_raw.airlines;

Query is running:   0%|          |

Downloading:   0%|          |

,id,name,alias,iata,icao,callsign,country,active,_data_source,_load_time
0,128,Alliance,None,None,AFJ,JAMBO,Uganda,N,openflights,2025-09-05 17:02:52.385895+00:00
1,465,Air Tanzania,None,TC,ATC,TANZANIA,Tanzania,Y,openflights,2025-09-05 17:02:52.385895+00:00
2,577,Alexandair,None,None,AXN,ALEXANDROS,Greece,N,openflights,2025-09-05 17:02:52.385895+00:00
3,647,Air Scorpio,None,None,SCU,SCORPIO UNIVERS,Bulgaria,N,openflights,2025-09-05 17:02:52.385895+00:00
4,1138,Air Parabet,None,None,PBT,PARABET,Bangladesh,N,openflights,2025-09-05 17:02:52.385895+00:00
...,...,...,...,...,...,...,...,...,...,...
6157,5126,Trans America,None,None,CLR,CLINTON AIRWAYS,United States,N,openflights,2025-09-05 17:02:52.385895+00:00
6158,5166,Trans Air,None,None,MUI,MAUI,United States,N,openflights,2025-09-05 17:02:52.385895+00:00
6159,5389,WSI Corporation,None,None,XWS,None,United States,N,openflights,2025-09-05 17:02:52.385895+00:00
6160,5434,Westward Airways,None,CN,WWD,WESTWARD,United States,N,openflights,2025-09-05 17:02:52.385895+00:00


##### Create staging table

In [ ]:
%%bigquery
create or replace table air_travel_stg.airlines as
  select airline_id as id,
  name,
  case alias
    when '\\N' then null
    else alias
    end as alias,
  case iata
    when '' then null
    else iata
    end as iata,
  case icao
    when '' then null
    else icao
    end as icao,
  case callsign
    when '\\N' then null
    else callsign
    end as callsign,
  case country
    when '\\N' then null
    else country end as country,
  active, _data_source, _load_time
  from air_travel_raw.airlines;

Query is running:   0%|          |

""


#### Check the output

In [ ]:
%%bigquery
select * from air_travel_stg.airlines
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,id,name,alias,iata,icao,callsign,country,active,_data_source,_load_time
0,3562,Multiflight,None,None,MFT,YORKAIR,None,N,openflights,2025-09-05 17:02:52.385895+00:00
1,5556,buzz,None,UK,BUZ,None,None,N,openflights,2025-09-05 17:02:52.385895+00:00
2,2636,Gulf African Airlines - Gambia,None,None,GUF,GULF AFRICAN,None,N,openflights,2025-09-05 17:02:52.385895+00:00
3,4854,Sirin,None,R1,None,None,None,N,openflights,2025-09-05 17:02:52.385895+00:00
4,4430,Scorpio Aviation,None,8S,None,None,None,N,openflights,2025-09-05 17:02:52.385895+00:00


### `airport_maps` table

#### Sample the raw table

In [ ]:
%%bigquery
select * from air_travel_raw.airport_maps
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,airport,terminal,business,category,location,dining,menu_items,_data_source,_load_time
0,cak,Terminal 1,Alamo Rent A Car,Car Rental,Near Baggage Claim,False,None,airport.guide,2025-09-12 22:27:13.846732+00:00
1,mob,Terminal 1,Chart Room Bar,Dining,"Level 2, Post-Security, near Hudson News",True,Bar beverages,airport.guide,2025-09-12 22:27:13.846732+00:00
2,lgb,Terminal 1,George's Greek Cafe,Dining,"North Concourse, near Gate G3",True,"Gyros, souvlaki, Greek salads, hummus",airport.guide,2025-09-12 22:27:13.846732+00:00
3,mli,Terminal 1,Charter Counter,Airline Service,"Pre-security ticketing area, next to Delta Air...",False,N/A,airport.guide,2025-09-12 22:27:13.846732+00:00
4,hou,Terminal 1,Space Corner,Retail,"West Concourse, near Gate 22",False,Not Applicable,airport.guide,2025-09-12 22:27:13.846732+00:00


In [ ]:
%%bigquery
select distinct category
from air_travel_raw.airport_maps
order by category

Query is running:   0%|          |

Downloading:   0%|          |

,category
0,Air Freight
1,Airline
2,Airline Check-in
3,Airline Gate / Check-in
4,Airline Lounge
5,Airline Service
6,Airline Services
7,Airline Ticketing
8,Airport Lounge
9,Airport Service


#### Standardize the category names

In [ ]:
%%bigquery
select airport, terminal, business,
      case category
        when 'Airline Services' then 'Airline Service'
        when 'Airline Services' then 'Airline Service'
        when 'Cafe' then 'Coffee Shop'
        when 'Financial Services' then 'Financial Service'
        when 'Retail/Dining' then 'Retail / Dining'
        when 'Things To Do' then 'Things to Do'
        when 'Transportation' then 'Transportation Service'
        when 'Vending' then 'Vending Machine'
        when 'Service' then 'Services'
        else category end as category,
      location, dining, menu_items, _data_source, _load_time
from air_travel_raw.airport_maps

Query is running:   0%|          |

Downloading:   0%|          |

,airport,terminal,business,category,location,dining,menu_items,_data_source,_load_time
0,cak,Terminal 1,Alamo Rent A Car,Car Rental,Near Baggage Claim,False,None,airport.guide,2025-09-12 22:27:13.846732+00:00
1,mob,Terminal 1,Chart Room Bar,Dining,"Level 2, Post-Security, near Hudson News",True,Bar beverages,airport.guide,2025-09-12 22:27:13.846732+00:00
2,lgb,Terminal 1,George's Greek Cafe,Dining,"North Concourse, near Gate G3",True,"Gyros, souvlaki, Greek salads, hummus",airport.guide,2025-09-12 22:27:13.846732+00:00
3,mli,Terminal 1,Charter Counter,Airline Service,"Pre-security ticketing area, next to Delta Air...",False,N/A,airport.guide,2025-09-12 22:27:13.846732+00:00
4,hou,Terminal 1,Space Corner,Retail,"West Concourse, near Gate 22",False,Not Applicable,airport.guide,2025-09-12 22:27:13.846732+00:00
...,...,...,...,...,...,...,...,...,...
1482,lax,1,Trejo's Tacos,Dining,Near Gate 13,True,"Tacos, burritos, bowls",airport.guide,2025-09-12 22:27:13.846732+00:00
1483,lax,3,Chicken + Beer,Dining,Near Gate 33,True,"Southern comfort food, fried chicken, craft beer",airport.guide,2025-09-12 22:27:13.846732+00:00
1484,sfo,Terminal 1,Goldilocks Filipino Cuisine,Dining,"Harvey Milk Terminal 1, Post-Security",True,Filipino fare,airport.guide,2025-09-12 22:27:13.846732+00:00
1485,sfo,Terminal 2,Andalé Mexican Kitchen + Bar,Dining,Concourse D,True,South-of-the-border fare,airport.guide,2025-09-12 22:27:13.846732+00:00


#### Create staging table

In [ ]:
%%bigquery
create or replace table air_travel_stg.airport_maps as
  select airport, terminal, business,
      case category
        when 'Airline Services' then 'Airline Service'
        when 'Airline Services' then 'Airline Service'
        when 'Cafe' then 'Coffee Shop'
        when 'Financial Services' then 'Financial Service'
        when 'Retail/Dining' then 'Retail / Dining'
        when 'Things To Do' then 'Things to Do'
        when 'Transportation' then 'Transportation Service'
        when 'Vending' then 'Vending Machine'
        when 'Service' then 'Services'
        else category end as category,
      location, dining, menu_items, _data_source, _load_time
from air_travel_raw.airport_maps

Query is running:   0%|          |

""


#### Check the output

In [ ]:
%%bigquery
select * from air_travel_stg.airport_maps
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,airport,terminal,business,category,location,dining,menu_items,_data_source,_load_time
0,las,Terminal 1,Jamba,Dining,B/C Connector,True,Smoothies,airport.guide,2025-09-12 22:27:13.846732+00:00
1,aus,Baggage Claim,Farmer's Fridge,Dining,Baggage Claim,True,"Fresh salads, bowls, and snacks from a vending...",airport.guide,2025-09-12 22:27:13.846732+00:00
2,aus,Baggage Claim,High Brew Coffee Kiosk,Dining,Baggage Claim,True,Canned cold brew coffee,airport.guide,2025-09-12 22:27:13.846732+00:00
3,fll,Terminal 2,Starbucks,Dining,Baggage Claim Level,True,"Coffee, Tea, Pastries, Sandwiches",airport.guide,2025-09-12 22:27:13.846732+00:00
4,fll,Terminal 4,Fresh Selections,Dining,Baggage Claim Level,True,"Grab-and-Go Sandwiches, Salads, Snacks",airport.guide,2025-09-12 22:27:13.846732+00:00


### `airport_reviews` table

In [ ]:
%%bigquery
select * from air_travel_raw.airport_reviews
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,id,threadRef,airportRef,airportIdent,date,memberNickname,subject,body,broad_category,sub_category,sentiment,language,_data_source,_load_time
0,233,206.0,1304,CNF4,2007-07-22 19:29:03,AnthonyNalli,Buttertarts,Famous for the best buttertarts anywhere!,Terminal Facilities,"Food, Beverage & Retail",Positive,English,"our-airports,llm",2025-09-19 18:37:01.277997+00:00
1,5461,4514.0,22661,N07,2010-04-05 19:08:48,None,Ed Gorski,"Wow, cant believe I found this sight. I worked...",Terminal Experience,Miscellaneous,Positive,English,"our-airports,llm",2025-09-19 18:37:01.277997+00:00
2,1772,1526.0,35150,KXSA,2008-05-21 8:05:24,david,New airport,This is a newly-built airport which opened in ...,Terminal Experience,Miscellaneous,Neutral,English,"our-airports,llm",2025-09-19 18:37:01.277997+00:00
3,4712,3897.0,2882,FKKD,2009-11-16 13:57:30,None,re: Cute Pure Bred English Bulldog Puppies,Im going through the same thing right with the...,Miscellaneous,Miscellaneous,Negative,English,"our-airports,llm",2025-09-19 18:37:01.277997+00:00
4,7809,6675.0,4533,LTBJ,2011-02-16 11:49:05,maverickk_85,LTBJ,ADNAN MENDERES HAVAALANI,Terminal Experience,Miscellaneous,Neutral,Turkish,"our-airports,llm",2025-09-19 18:37:01.277997+00:00


In [ ]:
%%bigquery
select date, safe_cast(split(date, ' ')[0] as DATE) as new_date
from air_travel_raw.airport_reviews
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,date,new_date
0,2007-07-22 19:29:03,2007-07-22
1,2010-04-05 19:08:48,2010-04-05
2,2008-05-21 8:05:24,2008-05-21
3,2009-11-16 13:57:30,2009-11-16
4,2011-02-16 11:49:05,2011-02-16


#### Cast `threadRef` and `date` and rename several  fields

In [ ]:
%%bigquery
select id,
	safe_cast(threadRef as INTEGER) as review_thread,
	airportRef as airport_id,
	airportIdent as airport_code,
	safe_cast(split(date, ' ')[0] as DATE) as review_date,
	memberNickname as review_author,
	subject as review_subject,
	body as review_details,
  broad_category,
  sub_category,
  sentiment,
  language,
	_data_source,
	_load_time
from air_travel_raw.airport_reviews

Query is running:   0%|          |

Downloading:   0%|          |

,id,review_thread,airport_id,airport_code,review_date,review_author,review_subject,review_details,broad_category,sub_category,sentiment,language,_data_source,_load_time
0,16964,14937,3384,KATL,2012-11-30,david,Easy international connection,"Nice, easy connection here in mid November -- ...",Terminal Experience,Connecting Flights,Positive,English,"our-airports,llm",2025-09-19 18:37:01.277997+00:00
1,21520,18879,3486,KDEN,2016-02-17,babtek,HUGE AIRPORT,But for connecting flights,Terminal Experience,Connecting Flights,Negative,English,"our-airports,llm",2025-09-19 18:37:01.277997+00:00
2,83819,81147,38380,YHDD,2022-01-21,adidas,incorrect position,incorrect position,Not Applicable,Not Applicable,Neutral,English,"our-airports,llm",2025-09-19 18:37:01.277997+00:00
3,83812,81140,351714,AU-0279,2022-01-21,adidas,MISSING FROM WEBSITE,MISSING FROM WEBSITE,Not Applicable,Not Applicable,Neutral,English,"our-airports,llm",2025-09-19 18:37:01.277997+00:00
4,21787,19146,27721,YLHI,2016-04-21,jwatsontravel,Easy flights if it's not raining,Flying from Port McQuarrie to Lord Howe Island...,Boarding & Departure,Deplaning & Arrival,Positive,English,"our-airports,llm",2025-09-19 18:37:01.277997+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14786,21559,18918,39793,CA-0147,2016-02-28,None,location,Where is it? Does not show up on google map.,Terminal Experience,Wayfinding & Information,Neutral,English,"our-airports,llm",2025-09-19 18:37:01.277997+00:00
14787,82744,80100,24375,S37,2020-07-24,wolfgangx,Agree on the hard to find,I also could not easily find this airport unti...,Terminal Experience,Wayfinding & Information,Negative,English,"our-airports,llm",2025-09-19 18:37:01.277997+00:00
14788,20773,18132,4885,MM37,2015-04-18,rstan,Not been here yet !!,Tried to plan a flight here but had to do lots...,Terminal Experience,Wayfinding & Information,Negative,English,"our-airports,llm",2025-09-19 18:37:01.277997+00:00
14789,18294,15811,2827,FAPP,2013-08-13,None,plane ticket and time of depature,Wanna know how much is the plane ticket and wh...,Terminal Experience,Wayfinding & Information,Neutral,English,"our-airports,llm",2025-09-19 18:37:01.277997+00:00


#### Create staging table

In [ ]:
%%bigquery
create or replace table air_travel_stg.airport_reviews as
	select id,
	safe_cast(threadRef as INTEGER) as review_thread,
	airportRef as airport_id,
	airportIdent as airport_code,
	safe_cast(split(date, ' ')[0] as DATE) as review_date,
	memberNickname as review_author,
	subject as review_subject,
	body as review_details,
	broad_category,
  sub_category,
  sentiment,
  language,
	_data_source,
	_load_time
	from air_travel_raw.airport_reviews

Query is running:   0%|          |

""


#### Check the output

In [ ]:
%%bigquery
select * from air_travel_stg.airport_reviews
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,id,review_thread,airport_id,airport_code,review_date,review_author,review_subject,review_details,broad_category,sub_category,sentiment,language,_data_source,_load_time
0,133,106,3483,KDCA,2007-07-18,XingR,Close In,One of the nation's better airports for the pa...,Transportation & Parking,Accessibility,Positive,English,"our-airports,llm",2025-09-19 18:37:01.277997+00:00
1,138,111,1926,CYTZ,2007-07-18,Bonnie,Awesome downtown airport!,This is a great airport if you are visiting To...,Transportation & Parking,Accessibility,Positive,English,"our-airports,llm",2025-09-19 18:37:01.277997+00:00
2,570,543,4468,LQMO,2007-08-04,None,Medugorje Mostar Airport,Mostar airport is the closest destination to M...,Transportation & Parking,Accessibility,Positive,English,"our-airports,llm",2025-09-19 18:37:01.277997+00:00
3,606,579,1991,CYZD,2007-08-08,david,Not open to public,"This airport is in an ideal location, right in...",Transportation & Parking,Accessibility,Neutral,English,"our-airports,llm",2025-09-19 18:37:01.277997+00:00
4,3371,2885,3622,KJFK,2009-01-30,ianturton,JKF Sucks,This has to be one of the worst airports it ha...,Terminal Experience,Accessibility,Negative,English,"our-airports,llm",2025-09-19 18:37:01.277997+00:00


### `airports` table



In [ ]:
%%bigquery
select * from air_travel_raw.airports
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,airport_id,airport_name,airport_city,airport_country,iata,icao,latitude,longitude,altitude,timezone,daylight_savings_time,tz_database_timezone,type,source,_data_source,_load_time
0,50,Arviat Airport,Eskimo Point,Canada,YEK,CYEK,61.094200,-94.070801,32,-6,A,America/Winnipeg,airport,OurAirports,openflights,2025-09-05 17:06:39.719594+00:00
1,80,Schefferville Airport,Schefferville,Canada,YKL,CYKL,54.805302,-66.805298,1709,-5,A,America/Toronto,airport,OurAirports,openflights,2025-09-05 17:06:39.719594+00:00
2,120,Regina International Airport,Regina,Canada,YQR,CYQR,50.431900,-104.666000,1894,-6,N,America/Regina,airport,OurAirports,openflights,2025-09-05 17:06:39.719594+00:00
3,153,Val-d'Or Airport,Val D'or,Canada,YVO,CYVO,48.053299,-77.782799,1107,-5,A,America/Toronto,airport,OurAirports,openflights,2025-09-05 17:06:39.719594+00:00
4,215,Ain Oussera Airport,Ain Oussera,Algeria,\N,DAAQ,35.525398,2.878710,2132,1,N,Africa/Algiers,airport,OurAirports,openflights,2025-09-05 17:06:39.719594+00:00


#### Profile `source` and `type` fields

In [ ]:
%%bigquery
select count(*) source_count
from air_travel_raw.airports
where source = '\\N'

Query is running:   0%|          |

Downloading:   0%|          |

,source_count
0,1651


In [ ]:
%%bigquery
select count(*) type_count
from air_travel_raw.airports
where type = '\\N'

Query is running:   0%|          |

Downloading:   0%|          |

,type_count
0,1651


In [ ]:
%%bigquery
	select safe_cast(airport_id as INTEGER) as id,
	airport_name as name,
	airport_city as city,
	airport_country as country,
	case iata
		when '\\N' then null
		else iata
		end as iata,
	case icao
		when '\\N' then null
		else icao
		end as icao,
	latitude,
	longitude,
	altitude,
	safe_cast(timezone as INTEGER) as timezone_delta,
  daylight_savings_time,
  case tz_database_timezone
		when '\\N' then null
		else tz_database_timezone
		end as timezone_name,
	case type
		when '\\N' then null
		else type
		end as type,
	case source
		when '\\N' then null
		else source
		end as source,
	_data_source
	_load_time
	from air_travel_raw.airports

Query is running:   0%|          |

Downloading:   0%|          |

,id,name,city,country,iata,icao,latitude,longitude,altitude,timezone_delta,daylight_savings_time,timezone_name,type,source,_load_time
0,1102,Amílcar Cabral International Airport,Amilcar Cabral,Cape Verde,SID,GVAC,16.741400,-22.949400,177,-1,U,Atlantic/Cape_Verde,airport,OurAirports,openflights
1,1105,Preguiça Airport,Sao Nocolau Island,Cape Verde,SNE,GVSN,16.588400,-24.284700,669,-1,U,Atlantic/Cape_Verde,airport,OurAirports,openflights
2,5674,Praia International Airport,"Praia, Santiago Island",Cape Verde,RAI,GVNP,14.924500,-23.493500,230,-1,U,Atlantic/Cape_Verde,airport,OurAirports,openflights
3,6436,Grytviken,Grytviken,South Georgia and the Islands,None,None,-54.165340,-36.302880,0,-2,N,Atlantic/South_Georgia,unknown,User,openflights
4,2252,Cassidy International Airport,Kiritimati,Kiribati,CXI,PLCH,1.986160,-157.350006,5,-12,U,None,airport,OurAirports,openflights
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12663,2661,Teniente Rodolfo Marsh Martin Base,Isla Rey Jorge,Antarctica,TNM,SCRM,-62.190800,-58.986698,147,12,U,Antarctica/South_Pole,airport,OurAirports,openflights
12664,7594,Tamana Island Airport,Tamana,Kiribati,TMN,NGTM,-2.485830,175.970001,0,12,U,Pacific/Tarawa,airport,OurAirports,openflights
12665,7605,Ebon Airport,Ebon,Marshall Islands,None,None,4.598889,168.753056,5,12,U,Pacific/Majuro,airport,User,openflights
12666,4077,Funafuti International Airport,Funafuti,Tuvalu,FUN,NGFU,-8.525000,179.195999,9,12,U,Pacific/Funafuti,airport,OurAirports,openflights


#### Create staging table

In [ ]:
%%bigquery
create or replace table air_travel_stg.airports as
	select safe_cast(airport_id as INTEGER) as id,
	airport_name as name,
	airport_city as city,
	airport_country as country,
	case iata
		when '\\N' then null
		else iata
		end as iata,
	case icao
		when '\\N' then null
		else icao
		end as icao,
	latitude,
	longitude,
	altitude,
	safe_cast(timezone as INTEGER) as timezone_delta,
  daylight_savings_time,
  case tz_database_timezone
		when '\\N' then null
		else tz_database_timezone
		end as timezone_name,
	case type
		when '\\N' then null
		else type
		end as type,
	case source
		when '\\N' then null
		else source
		end as source,
	_data_source,
	_load_time
	from air_travel_raw.airports

Query is running:   0%|          |

""


#### Check the output

In [ ]:
%%bigquery
select * from air_travel_stg.airports
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,id,name,city,country,iata,icao,latitude,longitude,altitude,timezone_delta,daylight_savings_time,timezone_name,type,source,_data_source,_load_time
0,12870,Homestead Inn Landing Field,Avery,United States,None,None,41.319790,-82.609369,656,-5,A,None,None,None,openflights,2025-09-05 17:06:39.719594+00:00
1,12542,Knivsta Railway Station,Knivsta,Sweden,None,None,59.725875,17.786375,61,1,E,None,None,None,openflights,2025-09-05 17:06:39.719594+00:00
2,12532,Eurotunnel Folkestone Terminal,Folkestone,United Kingdom,None,None,51.096000,1.135000,203,0,E,None,None,None,openflights,2025-09-05 17:06:39.719594+00:00
3,12866,Grosvenor Strathmore Metro Station,North Bethesda,United States,None,None,39.029188,-77.103904,280,-5,A,None,None,None,openflights,2025-09-05 17:06:39.719594+00:00
4,12447,Bernau bei Berlin Station,Bernau bei Berlin,Germany,None,None,52.675500,13.591600,237,1,E,None,None,None,openflights,2025-09-05 17:06:39.719594+00:00


### `countries` table

In [ ]:
%%bigquery
select *
from air_travel_raw.countries
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,country_name,iso_code,dafif_code,_data_source,_load_time
0,Argentina,AR,AR,openflights,2025-09-05 17:07:49.475555+00:00
1,Belarus,BY,BO,openflights,2025-09-05 17:07:49.475555+00:00
2,Eritrea,ER,ER,openflights,2025-09-05 17:07:49.475555+00:00
3,Guernsey,GG,GK,openflights,2025-09-05 17:07:49.475555+00:00
4,India,IN,IN,openflights,2025-09-05 17:07:49.475555+00:00


In [ ]:
%%bigquery
select count(*) as empty_iso_code
from air_travel_raw.countries
where iso_code = '\\N'

Query is running:   0%|          |

Downloading:   0%|          |

,empty_iso_code
0,19


In [ ]:
%%bigquery
select count(*) as empty_dafif_code
from air_travel_raw.countries
where dafif_code = '\\N'

Query is running:   0%|          |

Downloading:   0%|          |

,empty_dafif_code
0,0


##### Replace `'\\N'` with null and rename `country_name`

In [ ]:
%%bigquery
select country_name as name,
case iso_code
  when '\\N' then null
  else iso_code
  end as iso_code,
dafif_code
from air_travel_raw.countries

Query is running:   0%|          |

Downloading:   0%|          |

,name,iso_code,dafif_code
0,Argentina,AR,AR
1,Belarus,BY,BO
2,Eritrea,ER,ER
3,Guernsey,GG,GK
4,India,IN,IN
...,...,...,...
256,Brazil,BR,BR
257,Jersey,JE,JE
258,Jamaica,JM,JM
259,Malta,MT,MT


#### Create staging table

In [ ]:
%%bigquery
create or replace table air_travel_stg.countries as
    select country_name as name,
    case iso_code
      when '\\N' then null
      else iso_code
      end as iso_code,
    dafif_code,
    _data_source,
    _load_time
    from air_travel_raw.countries

Query is running:   0%|          |

""


#### Check the output

In [ ]:
%%bigquery
select * from air_travel_stg.countries
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,name,iso_code,dafif_code,_data_source,_load_time
0,Namibia,None,WA,openflights,2025-09-05 17:07:49.475555+00:00
1,Glorioso Islands,None,GO,openflights,2025-09-05 17:07:49.475555+00:00
2,Palmyra Atoll,None,LQ,openflights,2025-09-05 17:07:49.475555+00:00
3,Jarvis Island,None,DQ,openflights,2025-09-05 17:07:49.475555+00:00
4,Wake Island,None,WQ,openflights,2025-09-05 17:07:49.475555+00:00


### `flight_delays` table

In [ ]:
%%bigquery
select *
from air_travel_raw.flight_delays
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,_data_source,_load_time
0,2024,3,AS,Alaska Airlines Network,KOA,"Kona, HI: Ellison Onizuka Kona International a...",192.0,26.0,6.40,0.00,...,0.0,1.0,1062.0,304.0,0.0,521.0,0.0,237.0,on-time-performance,2025-09-05 17:00:47.734796+00:00
1,2024,2,9E,Endeavor Air Inc.,PNS,"Pensacola, FL: Pensacola International",33.0,3.0,1.45,0.00,...,1.0,0.0,240.0,93.0,0.0,0.0,0.0,147.0,on-time-performance,2025-09-05 17:00:47.734796+00:00
2,2024,2,NK,Spirit Airlines,RDU,"Raleigh/Durham, NC: Raleigh-Durham International",21.0,4.0,0.88,0.00,...,0.0,0.0,302.0,69.0,0.0,113.0,0.0,120.0,on-time-performance,2025-09-05 17:00:47.734796+00:00
3,2024,2,AS,Alaska Airlines Network,DTW,"Detroit, MI: Detroit Metro Wayne County",25.0,3.0,0.00,0.00,...,0.0,0.0,53.0,0.0,0.0,53.0,0.0,0.0,on-time-performance,2025-09-05 17:00:47.734796+00:00
4,2024,1,AS,Alaska Airlines Network,SLC,"Salt Lake City, UT: Salt Lake City International",104.0,37.0,13.41,0.25,...,14.0,0.0,2608.0,895.0,14.0,166.0,0.0,1533.0,on-time-performance,2025-09-05 17:00:47.734796+00:00


#### Split `airport_name` into three components: city, state, airport

In [ ]:
%%bigquery
select split(airport_name, ':')[0] as city_state, split(airport_name, ':')[1] as airport
from air_travel_raw.flight_delays
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,city_state,airport
0,"Kona, HI",Ellison Onizuka Kona International at Keahole
1,"Pensacola, FL",Pensacola International
2,"Raleigh/Durham, NC",Raleigh-Durham International
3,"Detroit, MI",Detroit Metro Wayne County
4,"Salt Lake City, UT",Salt Lake City International


In [ ]:
%%bigquery
select split(city_state, ',')[0] as city, split(city_state, ',')[1] as state, airport
from
  (select split(airport_name, ':')[0] as city_state, split(airport_name, ':')[1] as airport
  from air_travel_raw.flight_delays
  limit 5)

Query is running:   0%|          |

Downloading:   0%|          |

,city,state,airport
0,Kona,HI,Ellison Onizuka Kona International at Keahole
1,Pensacola,FL,Pensacola International
2,Raleigh/Durham,NC,Raleigh-Durham International
3,Detroit,MI,Detroit Metro Wayne County
4,Salt Lake City,UT,Salt Lake City International


#### Putting it all together

In [ ]:
%%bigquery
select event_month, carrier, carrier_name, airport_code,
  split(city_state, ',')[0] as airport_city, split(city_state, ',')[1] as airport_state,
  airport_name, * except (event_month, carrier, carrier_name, airport_code, airport_name, city_state)
from
  (select date(year, month, 01) as event_month, carrier, carrier_name, airport as airport_code,
  split(airport_name, ':')[0] as city_state, split(airport_name, ':')[1] as airport_name,
  safe_cast(arr_flights as INTEGER) as arr_total, safe_cast(arr_cancelled as INTEGER) as arr_cancelled,
  safe_cast(arr_diverted as INTEGER) as arr_diverted, safe_cast(arr_delay as INTEGER) as arr_delay_min,
  safe_cast(weather_delay as INTEGER) as weather_delay_min, safe_cast(nas_delay as INTEGER) as nas_delay_min,
  safe_cast(late_aircraft_delay as INTEGER) as late_aircraft_delay_min, _data_source, _load_time
  from air_travel_raw.flight_delays
  limit 5)

Query is running:   0%|          |

Downloading:   0%|          |

,event_month,carrier,carrier_name,airport_code,airport_city,airport_state,airport_name,arr_total,arr_cancelled,arr_diverted,arr_delay_min,weather_delay_min,nas_delay_min,late_aircraft_delay_min,_data_source,_load_time
0,2024-03-01,AS,Alaska Airlines Network,KOA,Kona,HI,Ellison Onizuka Kona International at Keahole,192,0,1,1062,0,521,237,on-time-performance,2025-09-05 17:00:47.734796+00:00
1,2024-02-01,9E,Endeavor Air Inc.,PNS,Pensacola,FL,Pensacola International,33,1,0,240,0,0,147,on-time-performance,2025-09-05 17:00:47.734796+00:00
2,2024-02-01,NK,Spirit Airlines,RDU,Raleigh/Durham,NC,Raleigh-Durham International,21,0,0,302,0,113,120,on-time-performance,2025-09-05 17:00:47.734796+00:00
3,2024-02-01,AS,Alaska Airlines Network,DTW,Detroit,MI,Detroit Metro Wayne County,25,0,0,53,0,53,0,on-time-performance,2025-09-05 17:00:47.734796+00:00
4,2024-01-01,AS,Alaska Airlines Network,SLC,Salt Lake City,UT,Salt Lake City International,104,14,0,2608,14,166,1533,on-time-performance,2025-09-05 17:00:47.734796+00:00


#### Create staging table

In [ ]:
%%bigquery
create or replace table air_travel_stg.flight_delays as
    select event_month, carrier, carrier_name, airport_code,
    split(city_state, ',')[0] as airport_city, split(city_state, ',')[1] as airport_state,
    airport_name, * except (event_month, carrier, carrier_name, airport_code, airport_name, city_state)
    from
        (select date(year, month, 01) as event_month, carrier, carrier_name, airport as airport_code,
          split(airport_name, ':')[0] as city_state, split(airport_name, ':')[1] as airport_name,
          safe_cast(arr_flights as INTEGER) as arr_total, safe_cast(arr_cancelled as INTEGER) as arr_cancelled,
          safe_cast(arr_diverted as INTEGER) as arr_diverted, safe_cast(arr_delay as INTEGER) as arr_delay_min,
          safe_cast(weather_delay as INTEGER) as weather_delay_min, safe_cast(nas_delay as INTEGER) as nas_delay_min,
          safe_cast(late_aircraft_delay as INTEGER) as late_aircraft_delay_min, _data_source, _load_time
          from air_travel_raw.flight_delays)

Query is running:   0%|          |

""


#### Check the output

In [ ]:
%%bigquery
select * from air_travel_stg.flight_delays
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,event_month,carrier,carrier_name,airport_code,airport_city,airport_state,airport_name,arr_total,arr_cancelled,arr_diverted,arr_delay_min,weather_delay_min,nas_delay_min,late_aircraft_delay_min,_data_source,_load_time
0,2003-09-01,TZ,ATA Airlines d/b/a ATA,ABQ,Albuquerque,NM,Albuquerque International Sunport,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,on-time-performance,2025-09-05 17:00:47.734796+00:00
1,2006-01-01,TZ,ATA Airlines d/b/a ATA,BWI,Baltimore,MD,Baltimore/Washington International Thurgood M...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,on-time-performance,2025-09-05 17:00:47.734796+00:00
2,2004-03-01,TZ,ATA Airlines d/b/a ATA,BWI,Baltimore,MD,Baltimore/Washington International Thurgood M...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,on-time-performance,2025-09-05 17:00:47.734796+00:00
3,2004-11-01,TZ,ATA Airlines d/b/a ATA,CLT,Charlotte,NC,Charlotte Douglas International,107,0,0,603,0,164,269,on-time-performance,2025-09-05 17:00:47.734796+00:00
4,2004-10-01,TZ,ATA Airlines d/b/a ATA,CLT,Charlotte,NC,Charlotte Douglas International,114,2,0,374,0,224,97,on-time-performance,2025-09-05 17:00:47.734796+00:00


### `flight_routes` table

In [ ]:
%%bigquery
select *
from air_travel_raw.flight_routes
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,airline_code,airline_id,source_airport,source_airport_id,dest_airport,dest_airport_id,codeshare,stops,equipment,_data_source,_load_time
0,2B,410,SVX,2975,KZN,2990,None,0,CR2,our-airports,2025-09-06 02:42:52.369196+00:00
1,2O,146,ADQ,3531,KKB,7168,None,0,BNI,our-airports,2025-09-06 02:42:52.369196+00:00
2,2Z,1729,BSB,2531,RAO,2613,None,0,AT7,our-airports,2025-09-06 02:42:52.369196+00:00
3,3G,595,DLA,897,DKR,1084,None,0,319,our-airports,2025-09-06 02:42:52.369196+00:00
4,3K,3021,SIN,3316,HKG,3077,None,0,320,our-airports,2025-09-06 02:42:52.369196+00:00


In [ ]:
%%bigquery
select count(*) as empty_source_airport_id
from air_travel_raw.flight_routes
where source_airport_id = '\\N'

Query is running:   0%|          |

Downloading:   0%|          |

,empty_source_airport_id
0,220


In [ ]:
%%bigquery
select count(*) as empty_dest_airport_id
from air_travel_raw.flight_routes
where dest_airport_id = '\\N'

Query is running:   0%|          |

Downloading:   0%|          |

,empty_dest_airport_id
0,221


#### Cast `airline_id`, replace `'\\N'` with null and rename airport fields

In [ ]:
%%bigquery
select airline_code,
  safe_cast(airline_id as INTEGER) as airline_id,
  source_airport as source_airport_code,
  case source_airport_id
    when '\\N' then null
    else safe_cast(source_airport_id as INTEGER)
    end as source_airport_id,
  dest_airport as dest_airport_code,
  case dest_airport_id
    when '\\N' then null
    else safe_cast(dest_airport_id as INTEGER)
    end as dest_airport_id,
  codeshare,
  stops,
  equipment,
  _data_source,
  _load_time
from air_travel_raw.flight_routes

Query is running:   0%|          |

Downloading:   0%|          |

,airline_code,airline_id,source_airport_code,source_airport_id,dest_airport_code,dest_airport_id,codeshare,stops,equipment,_data_source,_load_time
0,2B,410,SVX,2975,KZN,2990,None,0,CR2,our-airports,2025-09-06 02:42:52.369196+00:00
1,2O,146,ADQ,3531,KKB,7168,None,0,BNI,our-airports,2025-09-06 02:42:52.369196+00:00
2,2Z,1729,BSB,2531,RAO,2613,None,0,AT7,our-airports,2025-09-06 02:42:52.369196+00:00
3,3G,595,DLA,897,DKR,1084,None,0,319,our-airports,2025-09-06 02:42:52.369196+00:00
4,3K,3021,SIN,3316,HKG,3077,None,0,320,our-airports,2025-09-06 02:42:52.369196+00:00
...,...,...,...,...,...,...,...,...,...,...,...
67658,ZK,2607,DEN,3751,PIR,5761,None,0,BE1,our-airports,2025-09-06 02:42:52.369196+00:00
67659,ZK,2607,LBL,4341,DEN,3751,None,0,BE1 BEH,our-airports,2025-09-06 02:42:52.369196+00:00
67660,ZK,2607,MCK,7068,DEN,3751,None,0,BEH,our-airports,2025-09-06 02:42:52.369196+00:00
67661,ZL,4178,DBO,3358,SYD,3361,None,0,SF3,our-airports,2025-09-06 02:42:52.369196+00:00


#### Create staging table

In [ ]:
%%bigquery
create or replace table air_travel_stg.flight_routes as
  select airline_code,
    safe_cast(airline_id as INTEGER) as airline_id,
    source_airport,
    case source_airport_id
      when '\\N' then null
      else safe_cast(source_airport_id as INTEGER)
      end as source_airport_id,
    dest_airport,
    case dest_airport_id
      when '\\N' then null
      else safe_cast(dest_airport_id as INTEGER)
      end as dest_airport_id,
    codeshare,
    stops,
    equipment,
    _data_source,
    _load_time
  from air_travel_raw.flight_routes

Query is running:   0%|          |

""


#### Check the output

In [ ]:
%%bigquery
select * from air_travel_stg.flight_routes
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,airline_code,airline_id,source_airport,source_airport_id,dest_airport,dest_airport_id,codeshare,stops,equipment,_data_source,_load_time
0,2B,410,DME,4029,TGK,<NA>,None,0,CR2,our-airports,2025-09-06 02:42:52.369196+00:00
1,2G,1654,IKT,2937,KCK,<NA>,None,0,AN4,our-airports,2025-09-06 02:42:52.369196+00:00
2,4O,17885,MEX,1824,PQM,<NA>,None,0,SU9,our-airports,2025-09-06 02:42:52.369196+00:00
3,5Z,18946,CPT,797,PBZ,<NA>,None,0,BEH,our-airports,2025-09-06 02:42:52.369196+00:00
4,6E,2850,BOM,2997,HYD,<NA>,None,0,320,our-airports,2025-09-06 02:42:52.369196+00:00


### `tsa_traffic` table

In [ ]:
%%bigquery
select * from air_travel_raw.tsa_traffic
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,event_date,event_hour,airport_code,airport_name,airport_city,airport_state,security_checkpoint,total_traffic,_data_source,_load_time
0,11/19/2017,01:00,TPA,Tampa International,Tampa,FL,TPA C,0,tsa,2025-09-13 17:22:26.134702+00:00
1,11/19/2017,03:00,GSP,Greenville-Spartanburg,Green,SC,Checkpoint,147,tsa,2025-09-13 17:22:26.134702+00:00
2,11/19/2017,03:00,PHL,Philadelphia International,Philadelphia,PA,Checkpoint C,126,tsa,2025-09-13 17:22:26.134702+00:00
3,11/19/2017,03:00,SJC,San Jose International,San Jose,CA,Checkpoint B,14,tsa,2025-09-13 17:22:26.134702+00:00
4,11/19/2017,04:00,CHS,Charleston County International,Charleston,SC,Checkpoint,156,tsa,2025-09-13 17:22:26.134702+00:00


#### Cast `date` from STRING to DATE

In [ ]:
%%bigquery
select event_date as orig_date, safe_cast(event_date as DATE) as new_date
from air_travel_raw.tsa_traffic
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,orig_date,new_date
0,11/19/2017,NaT
1,11/19/2017,NaT
2,11/19/2017,NaT
3,11/19/2017,NaT
4,11/19/2017,NaT


In [ ]:
%%bigquery
select event_date as orig_date, safe_cast(event_date as DATE format 'MM/DD/YYYY') as new_date
from air_travel_raw.tsa_traffic
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,orig_date,new_date
0,11/19/2017,2017-11-19
1,11/19/2017,2017-11-19
2,11/19/2017,2017-11-19
3,11/19/2017,2017-11-19
4,11/19/2017,2017-11-19


#### Putting it all together

In [ ]:
%%bigquery
select safe_cast(event_date as DATE format 'MM/DD/YYYY') as event_date,
  safe_cast(event_hour as INTEGER) as event_hour,
  airport_code,
  airport_name,
  airport_city,
  airport_state,
  case security_checkpoint
    when 'Checkpoint' then null end as security_checkpoint,
  safe_cast(total_traffic as INTEGER) as total_traffic
from air_travel_raw.tsa_traffic
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,event_date,event_hour,airport_code,airport_name,airport_city,airport_state,security_checkpoint,total_traffic
0,2017-11-19,<NA>,TPA,Tampa International,Tampa,FL,<NA>,0
1,2017-11-19,<NA>,GSP,Greenville-Spartanburg,Green,SC,<NA>,147
2,2017-11-19,<NA>,PHL,Philadelphia International,Philadelphia,PA,<NA>,126
3,2017-11-19,<NA>,SJC,San Jose International,San Jose,CA,<NA>,14
4,2017-11-19,<NA>,CHS,Charleston County International,Charleston,SC,<NA>,156


#### Create staging table

In [ ]:
%%bigquery
create or replace table air_travel_stg.tsa_traffic as
  select safe_cast(event_date as DATE format 'MM/DD/YYYY') as event_date,
    safe_cast(event_hour as INTEGER) as event_hour,
    airport_code,
    airport_name,
    airport_city,
    airport_state,
    security_checkpoint,
    safe_cast(total_traffic as INTEGER) as total_traffic,
    _data_source,
    _load_time
  from air_travel_raw.tsa_traffic

Query is running:   0%|          |

""


#### Check the output

In [ ]:
%%bigquery
select * from air_travel_stg.tsa_traffic
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,event_date,event_hour,airport_code,airport_name,airport_city,airport_state,security_checkpoint,total_traffic,_data_source,_load_time
0,NaT,<NA>,None,None,None,None,None,223,tsa,2025-09-13 17:22:26.134702+00:00
1,2021-11-22,4,None,None,None,None,None,<NA>,tsa,2025-09-13 17:22:26.134702+00:00
2,2023-02-27,<NA>,None,None,None,None,None,<NA>,tsa,2025-09-13 17:22:26.134702+00:00
3,2021-07-29,<NA>,None,None,None,None,None,<NA>,tsa,2025-09-13 17:22:26.134702+00:00
4,NaT,<NA>,None,None,None,None,None,74,tsa,2025-09-13 17:22:26.134702+00:00


In [ ]:
%%bigquery
select * from air_travel_stg.tsa_traffic
where security_checkpoint is not null
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,event_date,event_hour,airport_code,airport_name,airport_city,airport_state,security_checkpoint,total_traffic,_data_source,_load_time
0,2021-11-22,<NA>,ORD,Chicago O'Hare International,Chicago,IL,0,1,tsa,2025-09-13 17:22:26.134702+00:00
1,2023-07-24,<NA>,ORD,Chicago O'Hare International,Chicago,IL,0,0,tsa,2025-09-13 17:22:26.134702+00:00
2,2024-02-12,<NA>,ORD,Chicago O'Hare International,Chicago,IL,0,47,tsa,2025-09-13 17:22:26.134702+00:00
3,2023-06-19,<NA>,ORD,Chicago O'Hare International,Chicago,IL,01,29,tsa,2025-09-13 17:22:26.134702+00:00
4,2023-06-19,<NA>,ORD,Chicago O'Hare International,Chicago,IL,01,153,tsa,2025-09-13 17:22:26.134702+00:00
